# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [21]:
#load all the imports 

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [13]:
#adjestment values | NEED TO CONFIGURE TO GET OUTPUT 

MAX_PRICE = 50.0      
MIN_RATING = 4        # Minimum rating (1-5)
NUM_PAGES = 20  

In [15]:
#builiding convertes for rating and price 

def convert_rating(rating_classes):
    """Convert rating class to number (1-5)"""
    rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    for rating in rating_classes:
        if rating in rating_map:
            return rating_map[rating]
    return 0

def convert_price(price_str):
    """Convert price string (£XX.XX) to float"""
    return float(price_str.replace('£', '').strip())

In [ ]:
### Main funciton to run webscrapting 

all_books = []

print(f"Scraping {NUM_PAGES} pages with criteria: Max £{MAX_PRICE}, Min {MIN_RATING} stars\n")

for page in range(1, NUM_PAGES + 1):
    print(f"Page {page}...", end=" ")
    
    # Get the page
    main_url = f"https://books.toscrape.com/catalogue/page-{page}.html"
    response = requests.get(main_url)
    html_code = BeautifulSoup(response.content, 'html.parser')
    
    # Find all books on the page
    book_grid = html_code.find("ol", attrs={"class": "row"})
    books = book_grid.find_all("li", attrs={"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"})
    
    page_count = 0
    
    for book in books:
        # Extract info from main page
        title_main = book.find("h3").get_text(strip=True)
        rating_main = convert_rating(book.find("p", attrs={"class": "star-rating"})["class"])
        price_main = convert_price(book.find("p", attrs={"class": "price_color"}).get_text().strip())
        
        # Get book URL
        image_container = book.find("div", attrs={"class": "image_container"})
        a_tag = image_container.find("a")
        url_book = a_tag['href']
        full_url = "https://books.toscrape.com/catalogue/" + url_book
        
        # Check if book meets criteria
        if price_main <= MAX_PRICE and rating_main >= MIN_RATING:
            # Go to book detail page
            book_response = requests.get(full_url)
            html_book_page = BeautifulSoup(book_response.content, 'html.parser')
            
            # Extract genre
            breadcrumb = html_book_page.find("ul", attrs={"class": "breadcrumb"})
            breadcrumb_items = breadcrumb.find_all("li")
            genre = breadcrumb_items[-2].get_text(strip=True)
            
            # Extract UPC, price, stock
            table = html_book_page.find("table", attrs={"class": "table table-striped"})
            td_items = table.find_all("td")
            
            upc = td_items[0].get_text(strip=True)
            price = td_items[3].get_text(strip=True)
            stock = td_items[5].get_text(strip=True)
            
            # Store all data
            all_books.append({
                'title': title_main,
                'upc': upc,
                'price': price_main,
                'stock': stock,
                'genre': genre,
                'rating': rating_main,
                'url': full_url
            })
            
            page_count += 1
            time.sleep(0.3)  # Be nice to the server
    
    print(f"Found {page_count} books")
    time.sleep(0.5)

Scraping 20 pages with criteria: Max £50.0, Min 4 stars

Page 1... Found 6 books
Page 2... Found 7 books
Page 3... Found 6 books
Page 4... Found 6 books
Page 5... Found 6 books
Page 6... Found 4 books
Page 7... Found 6 books
Page 8... Found 4 books
Page 9... Found 7 books
Page 10... Found 7 books
Page 11... Found 7 books
Page 12... Found 8 books
Page 13... Found 3 books
Page 14... Found 6 books
Page 15... Found 8 books
Page 16... Found 6 books
Page 17... Found 5 books
Page 18... Found 5 books
Page 19... Found 4 books
Page 20... Found 5 books


In [ ]:
#moving things to a DataFrame

df_books = pd.DataFrame(all_books)
df_books

,title,upc,price,stock,genre,rating,url
0,Sharp Objects,e00eb4fd7b871a48,47.82,In stock (20 available),Mystery,4,https://books.toscrape.com/catalogue/sharp-obj...
1,The Dirty Little Secrets ...,2597b5a345f45e1b,33.34,In stock (19 available),Business,4,https://books.toscrape.com/catalogue/the-dirty...
2,The Boys in the ...,e10e1e165dc8be4a,22.60,In stock (19 available),Default,4,https://books.toscrape.com/catalogue/the-boys-...
3,Shakespeare's Sonnets,30a7f60cd76ca58c,20.66,In stock (19 available),Poetry,4,https://books.toscrape.com/catalogue/shakespea...
4,Set Me Free,ce6396b0f23f6ecc,17.46,In stock (19 available),Young Adult,5,https://books.toscrape.com/catalogue/set-me-fr...
...,...,...,...,...,...,...,...
111,Furiously Happy: A Funny ...,c6a29d6df8de968c,41.46,In stock (12 available),Nonfiction,4,https://books.toscrape.com/catalogue/furiously...
112,Everyday Italian: 125 Simple ...,76319fe9d8b465a3,20.10,In stock (12 available),Food and Drink,5,https://books.toscrape.com/catalogue/everyday-...
113,Dirty (Dive Bar #1),df7e628bad5782b4,40.83,In stock (12 available),Romance,4,https://books.toscrape.com/catalogue/dirty-div...
114,A Paris Apartment,63343e287971bb0c,39.01,In stock (12 available),Historical Fiction,4,https://books.toscrape.com/catalogue/a-paris-a...


### Below is the breakdown of the input needed. 

In [2]:
#setup with main_url


main_url = "https://books.toscrape.com/catalogue/page-1.html" #page numbere will change from 1 - 50 

response = requests.get(main_url)
display(response)

htlm_code = BeautifulSoup(response.content) #readability and extracting data
book_grid = htlm_code.find("ol", attrs= {"class": "row"}) #find the book grid
books = book_grid.find_all("li", attrs= {"class":"col-xs-6 col-sm-4 col-md-3 col-lg-3"}) #view on individual books


<Response [200]>

In [3]:
#now in the main_url need to get the TITLE + PRICE & RATING 

#Title of the book
title_main = books[0].find("h3").get_text()

#Rating of the book
rating_main = books[0].find("p", attrs= {"class": "star-rating"})["class"]

#Price of the book
price_main = books[0].find("p", attrs= {"class": "price_color"}).get_text().strip()

#URL to book info
image_container = books[0].find("div", attrs= {"class": "image_container"})
if image_container:
    a_tag = image_container.find("a")  # Locate the <a> tag
    if a_tag and a_tag.has_attr('href'):
        url_book = a_tag['href']

        base_url = "https://books.toscrape.com/catalogue/"
        full_url = base_url + url_book


display(title_main)
display(rating_main)
display(price_main)
display(full_url)

'A Light in the ...'

['star-rating', 'Three']

'£51.77'

'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [4]:
### testing for book URL 

book_url = "https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html" 

response = requests.get(book_url)
display(response)

htlm_book_page = BeautifulSoup(response.content)



<Response [200]>

In [22]:
#Get the genre and title again from this page 
breadcrumb = htlm_book_page.find("ul", attrs= {"class" : "breadcrumb"})
breadcrumb_li_items = breadcrumb.find_all("li")


genre = breadcrumb_li_items[-2].get_text(strip=True)
title = breadcrumb_li_items[-1].get_text(strip=True)

#Get the UPC, Price, stock availability

table_striped = htlm_book_page.find("table", attrs={"class" : "table table-striped"})
table_striped_th_items = table_striped.find_all("td")

upc = table_striped_th_items[0].get_text()
price = table_striped_th_items[3].get_text()
stock = table_striped_th_items[-2].get_text()

display(upc)
display(price)
display(stock)
display(genre)
display(title)

'a897fe39b1053632'

'£51.77'

'In stock (22 available)'

'Poetry'

'A Light in the Attic'